# The very first Virtual Rat RNN

In [1]:
import numpy as np
from RNN import FirstRNN
from VirtualRatFunctions import *
from RNN_solver import RNNsolver
# auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

allRatsData = getData()

MySQLdb not found
Using mysql.connector
Loaded defaults from ~/.dbconf
Z009 (12771, 6)
A099 (23265, 6)
A109 (42333, 6)
Z014 (37888, 6)
Z010 (55682, 6)
A110 (27157, 6)
A112 (15870, 6)
A111 (56784, 6)
Z006 (40734, 6)
A117 (20919, 6)
A119 (29117, 6)
A121 (29079, 6)
A120 (18619, 6)
A106 (92159, 6)
A098 (68163, 6)


In [2]:
rat = allRatsData["Z009"]

x = np.zeros((1, rat.shape[0], 3))
y = np.zeros((1, rat.shape[0]))
trueY = np.zeros((1, rat.shape[0]))

x[0,:,:] = rat[:,:3]

train_num = int(rat.shape[0] * 0.8)
val_num = int(rat.shape[0] * 0.9)

trainX = x[:,:train_num,:]
valX = x[:,train_num:val_num,:]
testX = x[:,val_num:,:]

y[0,rat[:,3]>0] = 0
y[0,rat[:,4]>0] = 1
y[0,rat[:,5]>0] = 2

trainY = y[:,:train_num]
valY = y[:,train_num:val_num]
testY = y[:,val_num:]

trueY[0,:] = np.logical_not(np.bitwise_xor(rat[:,0],rat[:,1]))
trainTrueY = trueY[:,:train_num]
valTrueY = trueY[:,train_num:val_num]
testTrueY = trueY[:,val_num:]

In [33]:
print np.sum(valTrueY==0), np.sum(valTrueY==1), np.sum(valTrueY==2)
print np.sum(out==0), np.sum(out==1), np.sum(out==2)
print np.corrcoef(valY, out)


638 639 0
667 610 0
[[ 1.         -0.07912846]
 [-0.07912846  1.        ]]


In [38]:
RNN = FirstRNN(hidden_dim = 2)
solver = RNNsolver(RNN, trainX, trainTrueY,optim_config={
                 'learning_rate': 8e-5,
               }, num_epochs = 200,
                   verbose = True)
solver.train()
out = RNN.predict(valX)
acc = np.mean(out == valTrueY)
print acc


(Iteration 1 / 200) loss: 11514.403417
(Iteration 11 / 200) loss: 7461.996465
(Iteration 21 / 200) loss: 6895.439035
(Iteration 31 / 200) loss: 6449.462799
(Iteration 41 / 200) loss: 6079.562270
(Iteration 51 / 200) loss: 5763.690341
(Iteration 61 / 200) loss: 5456.233142
(Iteration 71 / 200) loss: 5081.872596
(Iteration 81 / 200) loss: 4195.309217
(Iteration 91 / 200) loss: 2322.417887
(Iteration 101 / 200) loss: 1216.350234
(Iteration 111 / 200) loss: 816.455215
(Iteration 121 / 200) loss: 613.365678
(Iteration 131 / 200) loss: 490.116195
(Iteration 141 / 200) loss: 407.258387
(Iteration 151 / 200) loss: 347.779952
(Iteration 161 / 200) loss: 303.070023
(Iteration 171 / 200) loss: 268.282364
(Iteration 181 / 200) loss: 240.476647
(Iteration 191 / 200) loss: 217.765003
1.0


In [39]:
for k, v in RNN.params.iteritems():
    print k, v

b [-1.61962402 -1.05825377]
b_vocab [ 3.40692353 -0.89586729 -2.51105785]
h0 [[ 0.00035516  0.00085488]]
W_vocab [[ 1.53145134 -3.34091663  0.38654137]
 [ 2.56627727 -2.37488461  0.90820515]]
Wh [[ 0.1030069  -0.08620285]
 [-0.18873303  0.20145527]]
Wx [[-2.92188621  2.44315553]
 [ 3.00621796 -2.80841804]
 [-0.78998059 -0.5990895 ]]


In [47]:
import helpers.DBUtilsClass as db
import cPickle as pkl

dbc = db.Connection()
D = {}
D['ratname'] = 'Rtnl'
D['train_size'] = train_num
D['test_size'] = val_num - train_num

D['loss'] = 217.765003
D['learning_rate'] = 8e-5
D['hidden_dim'] = 2
D['accuracy'] = 1.0
D['comments'] = "The well performing RNN with the smallest size ever."
D['b'] = pkl.dumps(RNN.params['b'])
D['b_vocab'] = pkl.dumps(RNN.params['b_vocab'])
D['W_vocab'] = pkl.dumps(RNN.params['W_vocab'])
D['h0'] = pkl.dumps(RNN.params['h0'])
D['Wh'] = pkl.dumps(RNN.params['Wh'])
D['Wx'] = pkl.dumps(RNN.params['Wx'])

dbc.saveToDB('vrat.rnn',D)

Loaded defaults from ~/.dbconf


0

In [19]:
lrs = np.random.uniform(1,100,20) * 1e-5
bestAcc = 0
bestRNN = None
bestLR = 0
for lr in lrs:
    RNN = FirstRNN(hidden_dim = 2)
    solver = RNNsolver(RNN, trainX, trainTrueY,optim_config={
                 'learning_rate': lr,
               }, verbose = False)
    solver.train()
    out = RNN.predict(valX)
    acc = np.mean(out == valTrueY)
    print "learning rate: %s, accuracy = %f" % (lr,acc)
    if acc > bestAcc:
        bestAcc = acc
        bestRNN = RNN
        bestLR = lr
        print "Best RNN so far: learning rate: %s, accuracy = %f" % (lr,acc)

learning rate: 0.000482546859735, accuracy = 0.495693
Best RNN so far: learning rate: 0.000482546859735, accuracy = 0.495693
learning rate: 0.000710451125549, accuracy = 0.500392
Best RNN so far: learning rate: 0.000710451125549, accuracy = 0.500392
learning rate: 0.000152714074241, accuracy = 0.660924
Best RNN so far: learning rate: 0.000152714074241, accuracy = 0.660924
learning rate: 0.000134399589891, accuracy = 0.742365
Best RNN so far: learning rate: 0.000134399589891, accuracy = 0.742365
learning rate: 0.000457100045655, accuracy = 0.618637
learning rate: 0.000559865787274, accuracy = 0.495693
learning rate: 0.0009430690382, accuracy = 0.500392
learning rate: 0.000650431005996, accuracy = 0.499608


KeyboardInterrupt: 